In [1]:
import scipy.io as sio
from scipy.fft import fft
import os,glob
import math
import numpy as np
import pandas as pd
import os,glob
import pyexcel
import signal_utll

In [2]:
df = pd.read_csv('../datasets/laser_ds.csv')
df.head()

,Age Group,Sex,Side,Sub,File
0,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...
1,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...
2,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...
3,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...
4,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...


In [4]:
Hz_Lthr, Hz_Hthr = 0.4, 3.5
mu = 1.65 # middle of .8 and 2.5
gamma=0.7
no_peaks = 3
top_peaks_array=[]
secs=60 # 1 mins

for i, row in df.iterrows():
    X, Y, peaks_idx = signal_utll.find_top_peaks(row['File'], Hz_Lthr, Hz_Hthr, mu, gamma, no_peaks)
    peaks_hr = [round(X[peak]*secs) for peak in peaks_idx]
    top_peaks_array.append(peaks_hr)

df['Peaks_Hr'] = top_peaks_array

In [5]:
df.head()

,Age Group,Sex,Side,Sub,File,Peaks_Hr
0,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...,"[97.0, 102.0, 105.0]"
1,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...,"[87.0, 105.0, 113.0]"
2,Age_A,Male,Neck_R,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_R_A...,"[93.0, 99.0, 101.0]"
3,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...,"[94.0, 96.0, 99.0]"
4,Age_A,Male,Neck_L,Sub 36,../datasets/Age_A/Age_A_Male/Sub 36/36Neck_L_A...,"[92.0, 100.0, 103.0]"


In [14]:
grouped_df = df.groupby(by=["Age Group","Sex","Sub"])

In [37]:
rows = []
for name, group in grouped_df: 
    n_row = list(name)
    gr_peaks = []
    for i, row in group.iterrows():
        if(len(row['Peaks_Hr']) == 3):
            gr_peaks.append(row['Peaks_Hr'])
    gr_peak = np.array(np.sum(gr_peaks, axis=0))/len(gr_peaks)
    gr_peak = [round(peak) for peak in gr_peak]
    gr_peak_min=min(gr_peak)
    gr_peak_max=max(gr_peak)
    n_row.append(gr_peak)
    n_row.append(gr_peak_min)
    n_row.append(gr_peak_max)
    rows.append(n_row)
    

df_Peaks= pd.DataFrame(rows);
df_Peaks

,0,1,2,3,4,5
0,Age_A,Female,Sub 25,"[96.0, 99.0, 103.0]",96.0,103.0
1,Age_A,Female,Sub 28,"[94.0, 100.0, 108.0]",94.0,108.0
2,Age_A,Female,Sub 32,"[90.0, 98.0, 108.0]",90.0,108.0
3,Age_A,Female,Sub 35,"[91.0, 99.0, 108.0]",91.0,108.0
4,Age_A,Female,Sub 44,"[84.0, 99.0, 110.0]",84.0,110.0
5,Age_A,Female,Sub 46,"[89.0, 100.0, 109.0]",89.0,109.0
6,Age_A,Female,Sub 63,"[91.0, 103.0, 115.0]",91.0,115.0
7,Age_A,Female,Sub 64,"[85.0, 103.0, 115.0]",85.0,115.0
8,Age_A,Female,Sub 66,"[82.0, 94.0, 107.0]",82.0,107.0
9,Age_A,Female,Sub 69,"[84.0, 96.0, 108.0]",84.0,108.0
